---
## Adım 1 için incelenecek kod aşağıdadır.Temel bir uygulamadır ve amaç neyle karşı karşıya olduğumuzu , amacımızın ne olduğunu anlamaktır

---

**H ve W eksenleri boyunca ayrı ayrı pooling yaparak konum bilgisini korur; bu iki eksen özetini ortak bir 1×1 dönüşümle birleştirip, tekrar iki ayrı attention (A_h ve A_w) üretir ve girdiyi X * A_h * A_w ile yeniden ölçekler. Kanal maliyetini de C → C/r reduction ile sınırlar**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0  # x * (Relu6(x+3) / 6)
# Swish in daha ucuz versiyonudur.[−3,+3] geçiş (non-linear) bölgesi. Bu aralıkta eğri “yumuşak” davranır; dışında ya 0’a yapışır ya lineer olur.
# .Matematiksel tanımı :
# ===>>> x * (Relu6(x+3) / 6) tür.O yüzden yukarıda relu6 dan HSwish fonksiyonunu üretilmiştir.

class CoordAtt(nn.Module):
    def __init__(
        self,
        in_channels, # Kanal sayısı
        reduction=32, # Boottleneck için sıkıştırma sayısı.Channels a bölünecektir.
        min_mid_channels=8, # Bottleneck için oluşturulan sıkıştırma sayısının kontrolüdür.
        act="hswish",
        alpha=1.0, # Attention ın emliyet kemeri.Attentionun ne kadar baskın olacağı kontrol edilir.
        learnable_alpha=False,
    ):
        super().__init__()
        mid = max(min_mid_channels, in_channels // reduction) # Ara katman hesaplaması

        self.conv1 = nn.Conv2d(in_channels, mid, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid) 
        # BN burada 1×1 conv çıktısını stabilize eder, sigmoid saturasyonunu azaltır, 
        # H/W kaynaklarının ölçeklerini dengeler. 

        if act.lower() == "hswish":
            self.act = HSwish()
        elif act.lower() == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act must be 'hswish' or 'relu'")

        self.conv_h = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True) 
        # Burdan kanal yada konum bilgisi işimize yarar mı onu seçiyoruz.
        # Height ve width için ayrı 1×1 head’ler; mid temsilden 
        # C kanala projekte edip A_h ve A_w maskelerini üretir (öğrenilebilir).

        self.conv_w = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True)

        if learnable_alpha:
            self.alpha = nn.Parameter(torch.tensor(float(alpha)))
        else:
            self.register_buffer("alpha", torch.tensor(float(alpha)))

        self._last_ah = None # Debug için...
        self._last_aw = None # Debug için...

    def forward(self, x):
        b, c, h, w = x.shape
        # Klasik bir CNN girdisi.Bunların dim karşılığı 
        # B = 0 ,C = 1 , H = 2 , W = 3 

        x_h = x.mean(dim=3, keepdim=True)  
        # İşlemin W uzayında yapılacağını söyler.W boyutunda ortalama alır 
        # Ortalaması alınan W uzayını kaldırır ve 1 koyar.
        # Son boyut = B,C,H,1

        x_w = x.mean(dim=2, keepdim=True).permute(0, 1, 3, 2)
        # İşlemin H uzayında yapılacağını belirtir.H uzanıyın ortalamasını alır.
        # Ortalaması alınan H uzayını kaldırır ve 1 koyar
        # Son boyut burda şudur :: :: ( B , C, 1, W)
        # İşlemlerin doğru olması için boyutları eşitlememiz gerekir.Bu işlemi permute yapar.
        # Boyutları indexlerine göre sıralatır.Ve son durumda boyut :: ( B,C,W,1) olur

        y = torch.cat([x_h, x_w], dim=2)
        # x_h: (B,C,H,1) ve x_w: (B,C,W,1) tensörlerini dim=2 boyunca uç uca ekler
        # Sonuç: (B,C,H+W,1)  -> H ve W özetleri tek eksende birleştirildi

        y = self.act(self.bn1(self.conv1(y))) 
        # cat sonrası 1×1 conv + BN + activation, height ve width eksenlerinden gelen konumsal 
        # özetlerin ayrılmadan önce ortak bir latent uzayda birlikte öğrenilmesini sağlar.

        y_h, y_w = torch.split(y, [h, w], dim=2)
        # Yukarıda concat + shared conv/BN/act sonrası oluşan ortak temsilden geçen mimari burda ayırılma işlemine geçer.
        # y_h değeri h , y_w değeri ise w yi temsil eder.
        # Bu işlemler yapılırken boyut :: ( B ,C ,H+W ,1) olduğundan dim = 2 seçilir.
        # y_h: (B, mid, H, 1)   (height tarafı)
        # y_w: (B, mid, W, 1)   (width tarafı)

        y_w = y_w.permute(0, 1, 3, 2)
        # Yukardaki 62.satırda yapılan permute işlemi tekrarlanır.
        # y_w değeri (B,mid,1,W) haline getirilir.
        # Bu noktadan sonra iki tensor ayrı kola gider.
        # Birisi W diğeri ise H ekseninde işlem yapar.Zaten cordinat attn de bunu ister.
        # y_h → H ekseninde değişen maske
        # y_w → W ekseninde değişen maske

        a_h = torch.sigmoid(self.conv_h(y_h))
        # H eksenindeki kanal burda ağırlıklandırılır
        # Height (H) ekseni boyunca kanal bazlı attention üretir.
        # conv_h: (B, mid, H, 1) -> (B, C, H, 1)
        # sigmoid ile ağırlıklar 0-1 aralığına sıkıştırılır (gate/ölçek katsayısı).
        
        a_w = torch.sigmoid(self.conv_w(y_w))
        # Yukarıdaki açıklamanın aynısı burda da H ekseni için yapılır.

        self._last_ah = a_h # DEBUG için değerler saklanır
        self._last_aw = a_w # DEBUG için değerler saklanır

        att = a_h * a_w # Çıkan değerler çarpılır ve attention artık hazır olur.

        scale = (1.0 - self.alpha) + self.alpha * att
        # Attention burda ölçeklenir.Ne kadar sert yada hafif olacağını ayarlar
        # Aynı zamanda attention burda kontrol edilir.Alpha değeri emniyet kemeri görevi görür.
        # Eğer alpha 0.8 ise :: 0.2 + 0.8*att değerinden attention ifadesi işleme alınır

        return x * scale # Modelin ileriye doğru hareketi için x ve ölçeklendirme çarpılır.

    @torch.no_grad()
# last_mask_stats(), son forward’daki a_h ve a_w attention maskelerinin 
# dağılımını (min/mean/max/std) görüp saturasyon veya aşırı bastırma gibi sorunları 
# erken yakalamak için kullanılan debug yardımcı fonksiyonudur.
    def last_mask_stats(self):
        if self._last_ah is None or self._last_aw is None:
            return None
        ah = self._last_ah
        aw = self._last_aw
        return {
            "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
            "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
        }

class ConvCoordAttBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        k=3,
        s=1,
        p=None,
        groups=1,
        norm="bn",
        act="silu",
        coord_reduction=32,
        coord_min_mid=8,
        coord_act="hswish",
        alpha=1.0,
        learnable_alpha=False,
    ):
        super().__init__()
        if p is None:
            p = k // 2

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=k, stride=s, padding=p, groups=groups, bias=False)

        if norm == "bn":
            self.norm = nn.BatchNorm2d(out_channels)
        elif norm == "gn":
            g = 8
            while out_channels % g != 0 and g > 1:
                g //= 2
            self.norm = nn.GroupNorm(g, out_channels)
        elif norm == "none":
            self.norm = nn.Identity()
        else:
            raise ValueError("norm must be 'bn', 'gn', or 'none'")

        if act == "silu":
            self.act = nn.SiLU(inplace=True)
        elif act == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act == "hswish":
            self.act = HSwish()
        else:
            raise ValueError("act must be 'silu', 'relu', or 'hswish'")

        self.att = CoordAtt(
            in_channels=out_channels,
            reduction=coord_reduction,
            min_mid_channels=coord_min_mid,
            act=coord_act,
            alpha=alpha,
            learnable_alpha=learnable_alpha,
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.act(x)
        return self.att(x)

@torch.no_grad()
def coordatt_debug_pass(module, x, name="CoordAttDebug"):
    module.eval()
    out = module(x)

    in_mean, in_std = float(x.mean()), float(x.std())
    out_mean, out_std = float(out.mean()), float(out.std())
    ratio = out_std / (in_std + 1e-12)

    print(f"\n=== {name} ===")
    print("in shape :", tuple(x.shape))
    print("out shape:", tuple(out.shape))
    print(f"in  mean/std : {in_mean:.6f} / {in_std:.6f}")
    print(f"out mean/std : {out_mean:.6f} / {out_std:.6f}")
    print(f"std_ratio(out/in): {ratio:.6f}")

    if hasattr(module, "last_mask_stats"):
        stats = module.last_mask_stats()
        if stats is not None:
            print("a_h stats:", stats["a_h"])
            print("a_w stats:", stats["a_w"])

    return out

def quick_sanity():
    torch.manual_seed(0)
    x = torch.randn(2, 64, 56, 56)

    att = CoordAtt(64, reduction=32, alpha=1.0, learnable_alpha=False)
    coordatt_debug_pass(att, x, "Only CoordAtt")

    blk = ConvCoordAttBlock(64, 64, k=3, s=1, norm="bn", act="silu", coord_reduction=32, alpha=1.0)
    coordatt_debug_pass(blk, x, "Conv + CoordAtt Block")

    blk.train()
    y = blk(x)
    loss = y.mean()
    loss.backward()
    print("\nBackward OK (loss:", float(loss), ")")

---
---
## Şimdi biraz bu kodun açıklamasına ve detaylarına inelim.Kodun üstünde yorum satırları zaten vardır.Ama daha detaylı anlatımları aşağıda bulabilirsiniz.
---
---


# HSwish (Hard-Swish) Aktivasyon Fonksiyonu

```python
class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0
```


**HSwish, Swish aktivasyon fonksiyonunun hesaplama açısından hafifletilmiş (approximation) halidir. Özellikle mobil ve hafif CNN mimarileri için tasarlanmıştır.**

### Tanım

##### Swish: Swish(x)=x⋅σ(x)

##### HSwish: HSwish(x)=x⋅(ReLU6(x+3)​) / 6

### Kod Karşılığı
```python
x * relu6(x + 3) / 6
```


Bu ifade:

* Sigmoid kullanmaz

* Parçalı doğrusal (piecewise linear) bir fonksiyondur

## Çalışma Mantığı

| x aralığı  | Davranış     |
| ---------- | ------------ |
| x ≤ -3     | çıktı = 0    |
| -3 < x < 3 | lineer geçiş |
| x ≥ 3      | çıktı = x    |


* Büyük pozitif değerleri aynen geçirir

* Büyük negatifleri bastırır

* Orta bölgede yumuşak geçiş sağlar

## ReLU ile Farkı

| Özellik                  | ReLU      | HSwish       |
| ------------------------ | --------- | ------------ |
| Negatif bölgede gradient | 0         | Kısmen var   |
| Yumuşaklık               | Keskin    | Daha yumuşak |
| Hesap maliyeti           | Çok düşük | Düşük        |
| Mobil uyum               | İyi       | **Çok iyi**  |


## CoordAtt İçinde Neden Tercih Edilir?

Coordinate Attention:

* Kanal + konum bilgisini birlikte işler

* Kanal sıkıştırma sonrası non-linearity gerektirir

HSwish:

* Stabil gradient sağlar

* Mobil dostudur

* 1×1 conv sonrası iyi çalışır

Bu nedenle CoordAtt ve MobileNetV3 tabanlı yapılarda varsayılan aktivasyon olarak kullanılır.

#### Özet olarak : **HSwish, Swish’in sigmoid içermeyen, ReLU6 tabanlı ve mobil için optimize edilmiş bir yaklaşık versiyonudur.**

----
---

# mid ne işe yarıyor?
```python
        mid = max(min_mid_channels, in_channels // reduction)
```


mid, şu iki işi aynı anda yapar:

### Kanal sıkıştırma (parametre ve FLOP kontrolü)
* Giriş kanalın C iken, ortak dönüşümde C → mid yaparsın.
* Bu sayede 1×1 conv’ların maliyeti düşer.

### Ortak bir gizli temsil (shared embedding) üretmek
* CoordAtt’te iki eksenden gelen özetler (x_h ve x_w) birleştiriliyor.
* Bu birleşik bilgi önce tek bir ortak projeksiyondan geçiyor:

**concat([x_h, x_w]) → shape (B, C, H+W, 1)**

**Conv1×1: C → mid → shape (B, mid, H+W, 1)**

* Bu mid uzayı, “yükseklik + genişlik” bilgisini tek bir latent uzayda karıştırıp temsil ediyor. Sonra bu latent temsil iki ayrı head ile tekrar C kanala genişletiliyor:

Conv_h: mid → C

Conv_w: mid → C

### Yani mid tam olarak şu rolü oynuyor:

**(H ve W) eksen bilgisini birleştiren ortak latent alan + maliyet düşüren bottleneck.**

## CBAM’deki hidden ile birebir benzerlik

CBAM-Channel (SE benzeri) içinde hidden şunun içindi:

* Kanal istatistiğini (C) düşük boyuta indirip ilişki öğrenmek

CoordAtt’te mid aynı şeyi yapıyor ama ek olarak:

* Girdi artık sadece C vektörü değil,

* (H+W) boyunca uzanan koordinat-encode edilmiş bir sinyal.
**Bu yüzden mid, hem kanal ilişkisi hem de eksensel konum bağlamını taşıyan ortak temsil oluyor.**

### Pratik sonuç: mid neyi değiştirir?

* mid küçükse: daha hafif, ama kapasite azalır → maskeler zayıflayabilir

* mid büyükse: daha güçlü, ama maliyet artar → overfit/over-suppress riski artabilir

**Bu yüzden mid = max(min_mid, C // reduction) dengesi standarttır.**

---
---

# 1×1 Conv sonrası neden normalization (BN) var?



```python
        self.conv1 = nn.Conv2d(in_channels, mid, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid)
```


## 1️⃣ 1×1 Conv burada ne yaptı?

Bu conv’un yaptığı şey şu:

* Girdi: (B, C, H+W, 1)

* Çıktı: (B, mid, H+W, 1)

Yani:

* Kanal sayısını C → mid indiriyor

* Height + Width eksenlerinden gelen bilgiyi lineer olarak karıştırıyor

Ama bu karıştırma kontrolsüz.

1×1 conv şunu yapar:

* 𝑦=Wx

Burada:

* x: concat edilmiş, istatistiği bozuk bir tensor

* W: rastgele başlatılmış ağırlıklar

Sonuç:

* Çıktının ölçeği garanti altında değil

## 2️⃣ 1×1 Conv çıkışı nasıl bir şey olur? (BN YOKSA)

BN yoksa şu problemler çıkar:

#### ❌ Problem 1: Ölçek patlar ya da çöker

Conv çıkışı:

* Çok büyük değerlere gidebilir

* Ya da çok küçük değerlere sıkışabilir

* Bu tensor birazdan sigmoid’e gidecek attention’ın atası.

BN yoksa:

* Attention maskeleri çok erken saturate olur

Model daha öğrenmenin başında:

* ya her şeyi açar , ya her şeyi kapatır

**Bu attention’ın agresifleşmesi dediğimiz durum.**

#### ❌ Problem 2: H ve W eksenlerinden biri domine eder

Unutma:

* x_h ve x_w farklı istatistiklere sahip

* Concat sonrası conv bunları karıştırıyor

BN yoksa:

* Conv, örneğin x_h tarafını çok büyütüp

* x_w tarafını bastırabilir (veya tam tersi)

Sonuç:

* Attention tek eksene kilitlenir

* CoordAtt’in temel fikri bozulur

#### ❌ Problem 3: Aktivasyon (HSwish) anlamsızlaşır

HSwish’in çalışma aralığı önemli:

* Etkili olduğu bölge ≈ [-3, 3]

BN yoksa:

* Conv çıkışı bu aralığın çok dışına çıkar

HSwish ya:

* tamamen lineer çalışır

* ya tamamen sıfır üretir

**Yani HSwish’i koymuş olsak bile fiilen ReLU gibi davranır.**

----
----

# Neden 1×1 conv’dan iki tane var? Neden biri H, biri W? Amaç ne?
```python
self.conv_h = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True)
self.conv_w = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True)
```


## 1️⃣ Bu noktaya gelene kadar elimizde ne var?

Şu anda elimizde:
```bash
y ∈ R^{B × mid × (H+W) × 1}
```
Bu tensor:

* Height ve Width bilgisini birlikte taşıyor

* Ortak bir latent uzayda (mid)

* Ama henüz attention değil

Yani:

**“Bu kanal + konum bilgisi önemli mi?”**
sorusunun ham cevabı.


----
----

## Sonrasında öğrenilebilir alpha koyduk.Bu alphayı parameter ile direkt modele verdik.Eğer false ise modele girmesin etkisi modele değimesin diye register buffer koyduk.Sonra da şu 2 değeri oluşturduk.Bunlar ne ?

```python
self._last_ah = None 
self._last_aw = None
```
**self._last_ah ve self._last_aw tamamen debug/izleme amaçlı iki değişken. Modelin “öğrenen” kısmı değiller; sadece son forward’da üretilen attention maskelerini sınıf içinde saklamak için varlar.**

----
----

# Şimdi ise forward kısmına geçelim.Bu satırlar ne işe yarıyor ?
```python
x_h = x.mean(dim=3, keepdim=True) 
x_w = x.mean(dim=2, keepdim=True).permute(0, 1, 3, 2) 
```
**Bu iki satır, Coordinate Attention’ın en temel hamlesi: 2D uzayı tek seferde yok etmek yerine, H ve W eksenlerini ayrı ayrı özetlemek.**

PyTorch’ta boyut indeksleri:

* dim=0 → B

* dim=1 → C

* dim=2 → H

* dim=3 → W

## 1) x_h = x.mean(dim=3, keepdim=True)

Ne yapıyor?
* Genişlik ekseni (W, dim=3) boyunca ortalama alıyor.

Anlamı:

* Her yükseklik satırı için (h),

* o satırdaki W boyunca ortalama aktivasyon.

**Yani: “Bu kanal, dikey eksende nerelerde güçlü?”**

## 2) x_w = x.mean(dim=2, keepdim=True).permute(0, 1, 3, 2)

İki adım var:
### 2.a) x.mean(dim=2, keepdim=True)

* Yükseklik ekseni (H, dim=2) boyunca ortalama alıyor.
* Bu aşamada shape: (B,C,1,W)

Anlamı:

* Her genişlik sütunu için (w),

* H boyunca ortalama aktivasyon.

**Yani: “Bu kanal, yatay eksende nerelerde güçlü?”**

### 2.b) .permute(0, 1, 3, 2)

Bu, boyutların yerini değiştiriyor:

- Önce: (B, C, 1, W)

- Sonra: (B, C, W, 1)

## Neden?
Çünkü birazdan x_h ile concat yapacağız:

* x_h: (B, C, H, 1)

* x_w (permute sonrası): (B, C, W, 1)

Bu ikisi artık aynı formatta:

* Son boyut 1

* Üçüncü boyut “uzunluk” (H veya W)

### Yani kısaca::
>x_h: W boyunca ortalama → (B,C,H,1) → height yönlü özet

>x_w: H boyunca ortalama → (B,C,1,W) sonra permute → (B,C,W,1) → width yönlü özet

>permute: concat için format eşleme (H ve W’yi aynı eksende yan yana koyabilmek için)

---
---

# 1.) cat burada ne yapıyor?
```python
y = torch.cat([x_h, x_w], dim=2)
```


* torch.cat, iki tensoru belirttiğin eksen (dim) boyunca uç uca ekler. Yani toplama değil; birleştirme.

Bizim elimizde iki tensor var:

* x_h shape: (B, C, H, 1)

* x_w shape: (B, C, W, 1) (permute sonrası)

Bu ikisini cat edebilmek için cat yapılmayan tüm boyutlar aynı olmalı:

* B aynı ✅

* C aynı ✅

* son boyut 1 aynı ✅

**sadece birleştireceğimiz boyut farklı: H ve W ✅**.Dolayısıyla bu kod sayesinde :
>Yani x_h ve x_w üçüncü boyutta yan yana dizildi.

# 2.) dim=2 ne işe yarıyor?

* dim=2 demek: 3. boyut (H ekseni slotu) boyunca birleştir demek.

PyTorch’ta indeksler:

* dim=0 → B

* dim=1 → C

* dim=2 → H (bizim burada “uzunluk ekseni” gibi kullanıyoruz)

* dim=3 → W veya 1

Biz x_w’yi permute ederek zaten (B,C,W,1) yaptık ki:

* dim=2 artık x_h için H

* dim=2 artık x_w için W

olacak şekilde aynı yere gelsin.

### **Neden dim=2 seçildi?**

Çünkü amacımız şuydu:

* Height ve Width özetlerini tek bir “sequence” gibi aynı eksende toplamak.

Bu yüzden:

* x_h’nin H uzunluğunu

- x_w’nin W uzunluğunu

aynı eksende topladık.

---
---
# Şimdi cat sonrası ifadeyi inceleyelim.
```python 
y_h, y_w = torch.split(y, [h, w], dim=2) 
y_w = y_w.permute(0, 1, 3, 2)
```

### 1) y_h, y_w = torch.split(y, [h, w], dim=2)

Ne yapıyor :: dim=2 eksenini (üçüncü boyutu) iki parçaya bölüyor:
* y_h = ilk h uzunluk

* y_w = sonraki w uzunluk

Yani:

* y_h artık height için ayrılmış latent temsil

* y_w artık width için ayrılmış latent temsil

### 2) y_w = y_w.permute(0, 1, 3, 2)
Şu anda y_w şu formatta:

* yw​ ∈R B×mid×W×1

Ama width attention üretmek için bizim hedef formatımız:

* (B×mid×1×W)

Çünkü birazdan şunu yapacağız:
```python
a_w = sigmoid(conv_w(y_w))
```
* ve bu a_w’nin şekli (B, C, 1, W) olmalı ki girdiyle broadcast düzgün çalışsın.

Bu yüzden permute(0,1,3,2) yapıyoruz:

* Önce: (B, mid, W, 1)

* Sonra: (B, mid, 1, W)

Yani son iki boyut yer değiştiriyor.

### Neden sadece y_w permute ediliyor?

Çünkü:

* y_h zaten (B, mid, H, 1) formatında ve height attention head bunu bekler.

* y_w ise concat öncesinde width tarafını (B, C, W, 1) formatına çevirmiştik.
* Şimdi tekrar width başlığının beklediği (B, mid, 1, W) formatına dönmemiz gerekiyor.

#### Genel özet olarak ::

**torch.split(y, [h, w], dim=2) işlemi, concat sonrası oluşan (H+W) uzunluk eksenini ilk H (height) ve sonraki W (width) parçalarına ayırarak y_h ∈ R^{B×mid×H×1} ve y_w ∈ R^{B×mid×W×1} üretir. Ardından y_w.permute(0,1,3,2) ile y_w (B×mid×1×W) formatına çevrilir; böylece width attention maskesi (B×C×1×W) üretmek ve girdiye doğru broadcast etmek mümkün olur.**

---
---

# Neden sigmoid?
```python
a_h = sigmoid(conv_h(y_h))  # (B, C, H, 1)
a_w = sigmoid(conv_w(y_w))  # (B, C, 1, W)  (öncesinde permute ile formatlanıyor)
```


Çünkü sigmoid çıktısı 0 ile 1 arasındadır:

* 0’a yakınsa → kapat (suppress)

* 1’e yakınsa → geçir (keep)

Bu maskeler sonra girdiye çarpılıyor:

* out = x * a_h * a_w

----
Buraya bir parantez açmak istiyorum.Permute işlemlerini daha net anlatabilmek için ufak bir notlama koyuyorum.Buna hem bu dosya içerisinden hemde aynı klasördeki permute.png görselinden ulaşabilirsiniz.

* x_h = mean over W → (B,C,H,1) (W silinir)

* x_w = mean over H → (B,C,1,W) (H silinir)

* x_w permute → (B,C,W,1) (cat için format)

* cat dim=2 → (B,C,H+W,1)

* split [H,W] → y_h:(B,mid,H,1), y_w:(B,mid,W,1)

* y_w permute back → (B,mid,1,W)

* conv_h/conv_w + sigmoid → a_h:(B,C,H,1), a_w:(B,C,1,W)

* out = x * a_h * a_w (broadcast)
----

---
---

# Artık son kısımdayız.(gating/mixing)
```python
att = a_h * a_w
scale = (1.0 - self.alpha) + self.alpha * att
return x * scale
```
### 1) att = a_h * a_w ne?

* a_h shape: (B, C, H, 1)

* a_w shape: (B, C, 1, W)

Broadcast ile çarpılınca: B×C×H×W

**Bu att bir attention maskesi: her kanal ve her konum için 0–1 arası (sigmoid’den geldiği için).**

### 2) scale neden böyle?
* scale=(1−α)+α⋅att

Bu şu demek:

* alpha = 1 ise: scale = att → klasik attention (direkt maske)

* alpha = 0 ise: scale = 1 → attention devre dışı (çıktı = x)

Yani bu formül aslında şunun aynısı:

* scale=1+α⋅(att−1)

**Attention’ın “1’den ne kadar saptığını” alpha ile ayarlıyoruz.**

### 3) Bu alpha niye kondu?

Çünkü attention blokları bazen:

* çok agresif davranır

* feature std’sini düşürür

* erken saturate edip öğrenmeyi zorlaştırır

Alpha:

* “attention gücü” için emniyet kemeri.

Bu, residual kapı mantığıyla aynı:

* alpha düşük → daha residual, daha güvenli

* alpha yüksek → daha attention odaklı, daha riskli ama daha etkili olabilir

### Küçük bir özet için :: :: :: 

**scale = (1-α) + α·att formülü attention’ı residual bir karışım olarak uygular; α=0 attention’ı kapatır, α=1 tam uygular, 0<α<1 etkisini yumuşatıp aşırı bastırma riskini azaltır (α>1 ise negatif ölçek riski doğurur).**

----


Yukarıda bulunan şu fonksiyon debug izlemek amacıyla yapılır.

```python 
@torch.no_grad()
def last_mask_stats(self):
    if self._last_ah is None or self._last_aw is None:
        return None
    ah = self._last_ah
    aw = self._last_aw
    return {
        "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
        "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
    }
```


Bu fonksiyon :: :: :: Forward içinde üretilen:

* a_h (B,C,H,1)

* a_w (B,C,1,W)

maskeleri self._last_ah ve self._last_aw içine kaydedilmişti.

**Bu fonksiyon da o maskelerin: min,mean,max,std istatistiklerini döndürüyor.**

-----

Diğer bir debug ve test fonksiyonu ise şudur:
```python 
@torch.no_grad()
def coordatt_debug_pass(module, x, name="CoordAttDebug"):
    module.eval()
    out = module(x)

    in_mean, in_std = float(x.mean()), float(x.std())
    out_mean, out_std = float(out.mean()), float(out.std())
    ratio = out_std / (in_std + 1e-12)

    print(f"\n=== {name} ===")
    print("in shape :", tuple(x.shape))
    print("out shape:", tuple(out.shape))
    print(f"in  mean/std : {in_mean:.6f} / {in_std:.6f}")
    print(f"out mean/std : {out_mean:.6f} / {out_std:.6f}")
    print(f"std_ratio(out/in): {ratio:.6f}")

    if hasattr(module, "last_mask_stats"):
        stats = module.last_mask_stats()
        if stats is not None:
            print("a_h stats:", stats["a_h"])
            print("a_w stats:", stats["a_w"])

    return out;
```
**Coordinate Attention bloğunun davranışını gözlemlemek ve doğrulamak için yazılmış bir yardımcı debug aracıdır. Eğitim sürecinin parçası değildir. Amaç, bloğun girişe kıyasla çıkışı ne kadar bastırdığını, boyutları koruyup korumadığını ve ürettiği attention maskelerinin sağlıklı dağılımlara sahip olup olmadığını hızlıca kontrol etmektir.**

**Fonksiyon, verilen bir girdiyi modülden geçirir ve giriş–çıkış arasındaki temel istatistikleri (ortalama, standart sapma) hesaplar. Özellikle çıkış standart sapmasının girişe oranı (std_ratio), attention’ın sinyali aşırı bastırıp bastırmadığını anlamak için kullanılır. Buna ek olarak, modül last_mask_stats metodunu destekliyorsa, height ve width yönlü attention maskelerinin minimum, ortalama, maksimum ve standart sapma değerlerini raporlar.**

**Özetle bu fonksiyon, Coordinate Attention bloğunun stabil, aşırı agresif olmayan ve beklenen şekilde çalışan bir attention üretip üretmediğini geliştirme ve test aşamasında doğrulamak için kullanılan saf bir debug yardımcı fonksiyonudur.**